# Installations

In [1]:
# %pip install "opencv-python-headless<4.3"
# %pip install cellpose
# %pip install cytoself

# %pip install adjustText

# %pip install --upgrade scikit-image

# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random
from model import load_data, get_analytics
from cytoself_custom import plot_umap, calc_umap_embvec, plot_feature_spectrum_from_image
from synthetic_multiplexing import multiplex
from metrics import plot_boostrapping, calc_bootstrapping, plot_metrics
from explainability import generate_images, plot_clusters_distance_heatmap
from utils import get_colors_dict
from sklearn.metrics.pairwise import euclidean_distances
import seaborn as sns

import os
import config
from config import SEED, PRETRAINED_MODEL_PATH, MODEL_PATH, METRICS_FOLDER, METRICS_MATCH_PATH, METRICS_RANDOM_PATH, MARKERS


np.random.seed(SEED)
random.seed(SEED)

%reload_ext autoreload
%autoreload 2
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [3]:
import logging
log_file_path = f"./logs/ranking_log.log"
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(levelname)s %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    handlers=[
        logging.FileHandler(log_file_path),
        logging.StreamHandler()
    ])

logging.info("init")

2022-12-22 11:08:02 INFO init


In [4]:
# neuroself_model_path = os.path.join("./models", "MODEL19_model_weights.0018.h5")
neuroself_model_path = os.path.join("./models", "MODEL18_model_weights.0040.h5")

# input_folders = ["./data/processed/220629_neurons_12markers_unstressed",
#                  "./data/processed/220707_neurons_12markers_stressed"]
input_folders = ["./data/processed/220814_neurons",
                 "./data/processed/220818_neurons",
                 "./data/processed/220831_neurons",
                 "./data/processed/220908",
                 "./data/processed/220914"]

# Figure 3

In [5]:
# fig_id = "Fig3"

# logging.info(f"[{fig_id}] init")

# # Load data
# logging.info(f"[{fig_id}] Load data")
# images, labels, labels_changepoints, markers_order = load_data(input_folders, markers=MARKERS, \
#                                                                 condition_l=True, type_l=False,\
#                                                                     split_by_set=True,
#                                                                     set_type = 'test',
#                                                                     cell_lines_include=["WT"],
#                                                                     split_by_set_include=[("WT", "unstressed")])

## D - Rank by condition

In [6]:
# panel_id = "D"

# logging.info(f"[{fig_id} {panel_id}] init")

# logging.info(f"[{fig_id} {panel_id}] get analytics_neuroself")
# analytics_neuroself = get_analytics(images, labels, model_path=neuroself_model_path)

# logging.info(f"[{fig_id} {panel_id}] Calculating embvec")
# analytics_neuroself.model.calc_embvec(analytics_neuroself.data_manager.test_data)
# X = analytics_neuroself.model.embvec[1]
# X = X.reshape(X.shape[0], -1)
# print(X.shape)


# # Setting clusters_centers as the median of all images of specific marker+cond
# logging.info(f"[{fig_id} {panel_id}] Calculating the median of each set of marker+cond combination")
# labels_s = pd.Series(labels.reshape(-1,))

# labels_unique = labels_s.unique()
# clusters_median = np.zeros((len(labels_unique), X.shape[1]))
# for j, l in enumerate(labels_unique):
#     ii = labels_s[labels_s.str.startswith(l)].index.tolist()
#     current_x = X[ii, :].copy()
#     clusters_median[j] = np.median(current_x, axis=0)

# # Calc dists
# logging.info(f"[{fig_id} {panel_id}] Calculating euclidean distances")
# dists = euclidean_distances(clusters_median)
# dists_df = pd.DataFrame(dists, index=labels_unique, columns=labels_unique)

# # Take only dists between the same marker in different conditions
# logging.info(f"[{fig_id} {panel_id}] Taking only the distances between conditions for each marker")
# dists_df_copy = dists_df.copy()
# markers = np.unique([m.split("_")[0] for m in dists_df_copy.index.to_list()])
# distances_for_marker = {m:0 for m in markers}
# dists_df_copy = dists_df_copy.loc[dists_df_copy.columns.str.contains(config.TERM_UNSTRESSED), dists_df_copy.columns.str.contains(config.TERM_STRESSED)] 

# for ind in dists_df_copy.index:
#     m, cond = ind.split('_')
#     print(m , cond)
#     cols_with_current_marker = dists_df_copy.columns.str.contains(f"{m}_")
#     distances_for_marker[m] = dists_df_copy.loc[ind,cols_with_current_marker]
    

# df_ranking = pd.DataFrame.from_dict(distances_for_marker, orient='index')
# df_ranking_indexes = df_ranking.index.tolist()
# df_ranking = df_ranking.to_numpy().diagonal()
# df_ranking = pd.Series(df_ranking, index=df_ranking_indexes)
# df_ranking.sort_values(ascending=False)

# Figure 4

In [7]:
fig_id = "Fig4"

## Ranking markers (ALS) (+plot)

In [8]:
fig_id = "Fig4"

logging.info(f"[{fig_id}] Init")

input_folders = [ "./data/processed/220814_neurons",
                 "./data/processed/220818_neurons",
                 "./data/processed/220831_neurons",
                 "./data/processed/220908",
                 "./data/processed/220914"]

neuroself_model_path = MODEL_PATH
groups_terms = [config.TERM_WT,config.TERM_TDP43, config.TERM_FUS, config.TERM_OPTN, config.TERM_TBK1]

# Load data
logging.info(f"[{fig_id}] Load data")
images, labels, labels_changepoints, markers_order = load_data(input_folders, markers=MARKERS, \
                                                                condition_l=False, type_l=True,\
                                                                    split_by_set=True,
                                                                    set_type = 'test',
                                                                    conds_include=["unstressed"],
                                                                    split_by_set_include=[("WT", "unstressed")])

2022-12-22 11:08:03 INFO [Fig4] Init
2022-12-22 11:08:03 INFO [Fig4] Load data


#########################################################
########### Splitting by set! (test) #############
#########################################################
Input folder: ./data/processed/220814_neurons
Input subfolder: CD41
Filepath: ./data/processed/220814_neurons/FUS/unstressed/CD41/220809_iNDI_FUS_unstressed-DAPI_CD41_NONO_TDP43-01_FUS.tif_processed
Input subfolder: CLTC
Filepath: ./data/processed/220814_neurons/FUS/unstressed/CLTC/220809_iNDI_FUS_unstressed-DAPI_Phalloidin_PSD95_CLTC-01_FUS.tif_processed
Input subfolder: DAPI
Filepath: ./data/processed/220814_neurons/FUS/unstressed/DAPI/220809_iNDI_FUS_unstressed-DAPI_CD41_NONO_TDP43-01_FUS.tif_processed
Filepath: ./data/processed/220814_neurons/FUS/unstressed/DAPI/220809_iNDI_FUS_unstressed-DAPI_G3BP1_TIA1_KIF5A-01_FUS.tif_processed
Filepath: ./data/processed/220814_neurons/FUS/unstressed/DAPI/220809_iNDI_FUS_unstressed-DAPI_PURA_SQSTM1_FMRP-01_FUS.tif_processed
Filepath: ./data/processed/220814_neurons/FUS/unstressed/D

### Old

In [9]:
panel_id = "Ranking markers (ALS)"

logging.info(f"[{fig_id} {panel_id}] get analytics_neuroself")
analytics_neuroself = get_analytics(images, labels, model_path=neuroself_model_path)

logging.info(f"[{fig_id} {panel_id}] Calculating embvec")
analytics_neuroself.model.calc_embvec(analytics_neuroself.data_manager.test_data)

X = analytics_neuroself.model.embvec[1]
X = X.reshape(X.shape[0], -1)
print(X.shape)


# Setting clusters_centers as the median of all images of specific marker+cond
logging.info(f"[{fig_id} {panel_id}] Calculating the median of each set of marker+cond combination")
labels_s = pd.Series(labels.reshape(-1,))

labels_unique = labels_s.unique()
clusters_median = np.zeros((len(labels_unique), X.shape[1]))
for j, l in enumerate(labels_unique):
    ii = labels_s[labels_s.str.startswith(l)].index.tolist()
    current_x = X[ii, :].copy()
    clusters_median[j] = np.median(current_x, axis=0)

# Calc dists
logging.info(f"[{fig_id} {panel_id}] Calculating euclidean distances")
dists = euclidean_distances(clusters_median)
dists_df = pd.DataFrame(dists, index=labels_unique, columns=labels_unique)

logging.info(f"[{fig_id} {panel_id}] Normalize values (min-max)")
dists_df_cols, dists_df_ind = dists_df.columns, dists_df.index
# To numpy for scaling accross all columns!
dists_df_np = dists_df.to_numpy()
dists_df_np_scaled = (dists_df_np-dists_df_np.min()) / (dists_df_np.max() - dists_df_np.min())
# Back to dataframe after scaling
dists_df_scaled = pd.DataFrame(dists_df_np_scaled, columns=dists_df_cols, index=dists_df_ind)

logging.info(f"[{fig_id} {panel_id}] Taking only the distances between cell lines for each marker")
dists_df_copy = dists_df_scaled.copy()
markers = np.unique([m.split("_")[0] for m in dists_df_copy.index.to_list()])
distances_for_marker_WT = {m:None for m in markers}
distances_for_marker_ALS = distances_for_marker_WT.copy()

for ind in dists_df_copy.index:
    m, cell_line = ind.split('_')
    logging.info(f"[{fig_id} {panel_id}] {m} {cell_line}")
    cols_with_current_cellline = dists_df_copy.columns.str.contains(f"{m}_{cell_line}")
    cols_with_current_marker = dists_df_copy.columns.str.contains(f"{m}_")
    if cell_line == "WT":
        vals = dists_df_copy.loc[cols_with_current_marker & cols_with_current_cellline,
                                 cols_with_current_marker & ~cols_with_current_cellline]
        distances_for_marker_WT[m] = vals.mean(axis=1).values[0]
    else:
        if distances_for_marker_ALS[m] is not None:
            continue
        cols_with_WT = dists_df_copy.columns.str.contains(f"{m}_WT")
        vals = dists_df_copy.loc[cols_with_current_marker & ~cols_with_WT,
                                 cols_with_current_marker & ~cols_with_WT]
        distances_for_marker_ALS[m] = np.sum(np.triu(vals.to_numpy())) * 1.0 / (vals.shape[0])
 
df_WT = pd.DataFrame.from_dict(distances_for_marker_WT, orient='index')
df_ALS = pd.DataFrame.from_dict(distances_for_marker_ALS, orient='index')

merged = pd.merge(df_WT, df_ALS, left_index=True, right_index=True)
merged.columns = ["WT", "ALS"]

logging.info(f"[{fig_id} {panel_id}] Sort values")
merged = merged.sort_values(by=["WT","ALS"], ascending=[False, True])
merged.to_csv("./ranking_mean.csv", index=True)

2022-12-22 11:11:57 INFO [Fig4 Ranking markers (ALS)] get analytics_neuroself


X, y:
(35670, 100, 100, 2) (35670,)
DAPI_OPTN         2797
DAPI_FUS          2706
DAPI_TBK1         2515
DAPI_TDP43        2344
Phalloidin_FUS     482
                  ... 
NEMO_WT             19
PURA_WT             18
FMRP_WT             18
SQSTM1_WT           18
TOMM20_WT           16
Length: 130, dtype: int64
Creating the model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2022-12-22 11:11:57 WARNING From /home/labs/hornsteinlab/sagyk/anaconda3/envs/cytoself/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2022-12-22 11:12:03 WARNING From /home/labs/hornsteinlab/sagyk/anaconda3/envs/cytoself/lib/python3.7/site-packages/tensorflow_core/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

2022-12-22 11:12:07.012955: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/share/lsf/10.1/linux3.10-glibc2.17-x86_64/lib
2022-12-22 11:12:07.012997: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-22 11:12:07.013028: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cn136.wexac.weizmann.ac.il): /proc/driver/nvidia/version does not exist
2022-12-22 11:12:07.013379: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2022-12-22 11:12:07.026027: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2022-12-22 11:12:07.026244: I tensorflow/compiler/xla/

./models/MODEL19_model_weights.0018.h5
is loaded.
Ground truth:
['ANXA11_FUS' 'ANXA11_OPTN' 'ANXA11_TBK1' 'ANXA11_TDP43' 'ANXA11_WT'
 'CD41_FUS' 'CD41_OPTN' 'CD41_TBK1' 'CD41_TDP43' 'CD41_WT' 'CLTC_FUS'
 'CLTC_OPTN' 'CLTC_TBK1' 'CLTC_TDP43' 'CLTC_WT' 'Calreticulin_FUS'
 'Calreticulin_OPTN' 'Calreticulin_TBK1' 'Calreticulin_TDP43'
 'Calreticulin_WT' 'DAPI_FUS' 'DAPI_OPTN' 'DAPI_TBK1' 'DAPI_TDP43'
 'DAPI_WT' 'DCP1A_FUS' 'DCP1A_OPTN' 'DCP1A_TBK1' 'DCP1A_TDP43' 'DCP1A_WT'
 'FMRP_FUS' 'FMRP_OPTN' 'FMRP_TBK1' 'FMRP_TDP43' 'FMRP_WT' 'FUS_FUS'
 'FUS_OPTN' 'FUS_TBK1' 'FUS_TDP43' 'FUS_WT' 'G3BP1_FUS' 'G3BP1_OPTN'
 'G3BP1_TBK1' 'G3BP1_TDP43' 'G3BP1_WT' 'GM130_FUS' 'GM130_OPTN'
 'GM130_TBK1' 'GM130_TDP43' 'GM130_WT' 'KIF5A_FUS' 'KIF5A_OPTN'
 'KIF5A_TBK1' 'KIF5A_TDP43' 'KIF5A_WT' 'LAMP1_FUS' 'LAMP1_OPTN'
 'LAMP1_TBK1' 'LAMP1_TDP43' 'LAMP1_WT' 'NEMO_FUS' 'NEMO_OPTN' 'NEMO_TBK1'
 'NEMO_TDP43' 'NEMO_WT' 'NONO_FUS' 'NONO_OPTN' 'NONO_TBK1' 'NONO_TDP43'
 'NONO_WT' 'Nucleolin_FUS' 'Nucleolin_OPTN' 'Nucleo

2022-12-22 11:41:15 INFO [Fig4 Ranking markers (ALS)] Calculating the median of each set of marker+cond combination


(35670, 9216)


2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] Calculating euclidean distances
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] Normalize values (min-max)
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] Taking only the distances between cell lines for each marker
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] CD41 FUS
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] CLTC FUS
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] DAPI FUS
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] FMRP FUS
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] G3BP1 FUS
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] KIF5A FUS
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] NONO FUS
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] PSD95 FUS
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] PURA FUS
2022-12-22 11:41:23 INFO [Fig4 Ranking markers (ALS)] Phalloidin FUS
2022-12-22 11:41:24 INFO [Fig4 Ranking markers (ALS)] SQSTM1 FUS
2022-12-22 11:

In [10]:
# # Plot
# logging.info(f"[{fig_id} {panel_id}] Normalize values (standard scaler)")
# merged_norm = pd.DataFrame([(merged[col]-merged[col].mean())/merged[col].std() for col in merged.columns]).T

# logging.info(f"[{fig_id} {panel_id}] Plot")
# plt.scatter(merged_norm.loc[:,'WT'], merged_norm.loc[:,'ALS'])
# for i in range(merged_norm.shape[0]):
#     plt.annotate(merged_norm.index[i], (merged_norm.iloc[i]['WT'], merged_norm.iloc[i]['ALS']))
# plt.xlabel('WT')
# plt.ylabel('ALS')
# plt.show()

### New

In [11]:
fig_id = "Fig4"

logging.info(f"[{fig_id}] Init")

input_folders = [ "./data/processed/220814_neurons",
                 "./data/processed/220818_neurons",
                 "./data/processed/220831_neurons",
                 "./data/processed/220908",
                 "./data/processed/220914"]

neuroself_model_path = MODEL_PATH
groups_terms = [config.TERM_WT,config.TERM_TDP43, config.TERM_FUS, config.TERM_OPTN, config.TERM_TBK1]

# Load data
logging.info(f"[{fig_id}] Load data")
images, labels, labels_changepoints, markers_order = load_data(input_folders, markers=MARKERS, \
                                                                condition_l=False, type_l=True,\
                                                                    split_by_set=True,
                                                                    set_type = 'test',
                                                                    conds_include=["unstressed"],
                                                                    split_by_set_include=[("WT", "unstressed")])

2022-12-22 11:41:24 INFO [Fig4] Init
2022-12-22 11:41:24 INFO [Fig4] Load data


#########################################################
########### Splitting by set! (test) #############
#########################################################
Input folder: ./data/processed/220814_neurons
Input subfolder: CD41
Filepath: ./data/processed/220814_neurons/FUS/unstressed/CD41/220809_iNDI_FUS_unstressed-DAPI_CD41_NONO_TDP43-01_FUS.tif_processed
Input subfolder: CLTC
Filepath: ./data/processed/220814_neurons/FUS/unstressed/CLTC/220809_iNDI_FUS_unstressed-DAPI_Phalloidin_PSD95_CLTC-01_FUS.tif_processed
Input subfolder: DAPI
Filepath: ./data/processed/220814_neurons/FUS/unstressed/DAPI/220809_iNDI_FUS_unstressed-DAPI_CD41_NONO_TDP43-01_FUS.tif_processed
Filepath: ./data/processed/220814_neurons/FUS/unstressed/DAPI/220809_iNDI_FUS_unstressed-DAPI_G3BP1_TIA1_KIF5A-01_FUS.tif_processed
Filepath: ./data/processed/220814_neurons/FUS/unstressed/DAPI/220809_iNDI_FUS_unstressed-DAPI_PURA_SQSTM1_FMRP-01_FUS.tif_processed
Filepath: ./data/processed/220814_neurons/FUS/unstressed/D

In [12]:
panel_id = "Ranking by cell line"

logging.info(f"[{fig_id} {panel_id}] get analytics_neuroself")
analytics_neuroself = get_analytics(images, labels, model_path=neuroself_model_path)
logging.info(f"[{fig_id} {panel_id}] Calculating embvec")
analytics_neuroself.model.calc_embvec(analytics_neuroself.data_manager.test_data)

logging.info(f"[{fig_id} {panel_id}] Init")
X = analytics_neuroself.model.embvec[1]
X = X.reshape(X.shape[0], -1)
print(X.shape)


# Setting clusters_centers as the median of all images of specific marker+cond
logging.info(f"[{fig_id} {panel_id}] Calculating the median of each set of marker+cond combination")
labels_s = pd.Series(labels.reshape(-1,))

labels_unique = labels_s.unique()
clusters_median = np.zeros((len(labels_unique), X.shape[1]))
for j, l in enumerate(labels_unique):
    ii = labels_s[labels_s.str.startswith(l)].index.tolist()
    current_x = X[ii, :].copy()
    clusters_median[j] = np.median(current_x, axis=0)

# Calc dists
logging.info(f"[{fig_id} {panel_id}] Calculating euclidean distances")
dists = euclidean_distances(clusters_median)
dists_df = pd.DataFrame(dists, index=labels_unique, columns=labels_unique)

logging.info(f"[{fig_id} {panel_id}] Normalize values (min-max)")
dists_df_cols, dists_df_ind = dists_df.columns, dists_df.index
# To numpy for scaling accross all columns!
dists_df_np = dists_df.to_numpy()
dists_df_np_scaled = (dists_df_np-dists_df_np.min()) / (dists_df_np.max() - dists_df_np.min())
# Back to dataframe after scaling
dists_df_scaled = pd.DataFrame(dists_df_np_scaled, columns=dists_df_cols, index=dists_df_ind)

logging.info(f"[{fig_id} {panel_id}] Taking only the distances between cell lines for each marker")
markers = np.unique([m.split("_")[0] for m in dists_df.index.to_list()])

wt_hits = dists_df_scaled.index.str.contains(config.TERM_WT)
for line in [config.TERM_TDP43, config.TERM_FUS, config.TERM_OPTN, config.TERM_TBK1]:
    line_hits = dists_df_scaled.columns.str.contains(line)
    rankings = dists_df_scaled.loc[wt_hits, line_hits]
    rankings = rankings.loc[sorted(rankings.index), sorted(rankings.columns)]
    rankings = pd.Series(np.diag(rankings), index=rankings.index).sort_values(ascending=False)
    rankings.to_csv(f"./ranking_{line}.csv", index=True)
    logging.info(line)
    logging.info(rankings)
    logging.info('----')

# plt.figure(figsize=(20,20))
# sns.clustermap(dists_df_scaled, yticklabels=True, xticklabels=True)
# plt.show()

# for m in markers:
#     logging.info(f"[{fig_id} {panel_id}] {m}")
#     cols_with_current_marker = dists_df_scaled.columns.str.contains(f"{m}_")
#     d = dists_df_scaled.loc[cols_with_current_marker, cols_with_current_marker]
#     d = d.sort_index(axis=0).sort_index(axis=1)
#     sns.heatmap(d)
#     plt.title(m)
#     plt.show()


2022-12-22 11:46:34 INFO [Fig4 Ranking by cell line] get analytics_neuroself


X, y:
(35670, 100, 100, 2) (35670,)
DAPI_OPTN         2797
DAPI_FUS          2706
DAPI_TBK1         2515
DAPI_TDP43        2344
Phalloidin_FUS     482
                  ... 
NEMO_WT             19
PURA_WT             18
FMRP_WT             18
SQSTM1_WT           18
TOMM20_WT           16
Length: 130, dtype: int64
Creating the model


All files will be saved at model_outputs/


Loading weights
./models/MODEL19_model_weights.0018.h5
is loaded.
Ground truth:
['ANXA11_FUS' 'ANXA11_OPTN' 'ANXA11_TBK1' 'ANXA11_TDP43' 'ANXA11_WT'
 'CD41_FUS' 'CD41_OPTN' 'CD41_TBK1' 'CD41_TDP43' 'CD41_WT' 'CLTC_FUS'
 'CLTC_OPTN' 'CLTC_TBK1' 'CLTC_TDP43' 'CLTC_WT' 'Calreticulin_FUS'
 'Calreticulin_OPTN' 'Calreticulin_TBK1' 'Calreticulin_TDP43'
 'Calreticulin_WT' 'DAPI_FUS' 'DAPI_OPTN' 'DAPI_TBK1' 'DAPI_TDP43'
 'DAPI_WT' 'DCP1A_FUS' 'DCP1A_OPTN' 'DCP1A_TBK1' 'DCP1A_TDP43' 'DCP1A_WT'
 'FMRP_FUS' 'FMRP_OPTN' 'FMRP_TBK1' 'FMRP_TDP43' 'FMRP_WT' 'FUS_FUS'
 'FUS_OPTN' 'FUS_TBK1' 'FUS_TDP43' 'FUS_WT' 'G3BP1_FUS' 'G3BP1

2022-12-22 11:46:49 INFO [Fig4 Ranking by cell line] Calculating embvec


Inferring embedding vectors...
35670/35670 [==============================] - 1744s 49ms/sample


2022-12-22 12:15:53 INFO [Fig4 Ranking by cell line] Init
2022-12-22 12:15:53 INFO [Fig4 Ranking by cell line] Calculating the median of each set of marker+cond combination


(35670, 9216)


2022-12-22 12:16:01 INFO [Fig4 Ranking by cell line] Calculating euclidean distances
2022-12-22 12:16:01 INFO [Fig4 Ranking by cell line] Normalize values (min-max)
2022-12-22 12:16:01 INFO [Fig4 Ranking by cell line] Taking only the distances between cell lines for each marker
2022-12-22 12:16:01 INFO _TDP43
2022-12-22 12:16:01 INFO FMRP_WT            0.742645
TDP43_WT           0.735106
PSD95_WT           0.692609
mitotracker_WT     0.684843
Nucleolin_WT       0.681803
FUS_WT             0.618645
CLTC_WT            0.615797
SQSTM1_WT          0.589607
NONO_WT            0.582082
PURA_WT            0.574192
CD41_WT            0.565670
TOMM20_WT          0.557110
PML_WT             0.495424
ANXA11_WT          0.446820
TIA1_WT            0.431798
LAMP1_WT           0.421030
KIF5A_WT           0.417707
SNCA_WT            0.414593
Phalloidin_WT      0.412751
PEX14_WT           0.407554
GM130_WT           0.406129
G3BP1_WT           0.394273
NEMO_WT            0.373724
Calreticulin_WT    0

In [13]:
logging.info(f"DONE!")

2022-12-22 12:16:02 INFO DONE!
